# Transit Boardings Report
This report includes two types of report:
- Standard: A single or comparison report for the overall scenario(s)
    - This includes sub-mode daily boardings and TOD totals (AM, MD, PM, NT) 
- Detailed: A detailed report for selected links.
    - This includes Daily and TOD boardings for user specified route(s)


In [ ]:
#import required libraries
import pandas as pd
import numpy as np
import os
import glob
from functools import reduce
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly import tools

In [ ]:
#one or two scenarios?
bases = {'Base Model':r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2040 NB Scen 01_MoDXoutputs/',
    'Comparative Model':r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2016 Scen 00_08March2019_MoDXoutputs/'} #MAX 2 BASES

#standard or detailed?
routes = r'G:/Data_Resources/DataStore/transit info.csv'
    #detailed can include multiple routes
routefile = pd.read_csv(routes)
routeList = routefile['Route_ID'] #[7887,8188,7888,8187,8339,8386,8387,8388]


In [ ]:
#import transit assignment results
def import_transit_assignment(b):
    '''bring in data and combine into sum tables for daily and put into a dictionary'''
    base = b+r'out/'
    tod = ["AM/", "MD/", "PM/", "NT/"]
    TODsums = {} # this will eventually contain all the TOD summed results

    #import csv and create sum tables for all TOD and for all day
    for x in tod:
        paths = glob.glob(os.path.join(base,x,r'*.csv'))
        tablist = []
        for p in paths:
            tablist.append(pd.read_csv(p).set_index(['ROUTE','STOP'])) #read in, set indices, and append to list
        if len(routeList) > 0: #deal with detailed filtering if needed
            for t in range(len(tablist)):
                tablist[t] = tablist[t][tablist[t].index.get_level_values('ROUTE').isin(routeList)]
        TODsums[x[0:2]] = reduce(lambda a, b: a.add(b, fill_value=0), tablist) #sum the tables for the TOD
    TODsums['daily'] = reduce(lambda a, b: a.add(b, fill_value=0), TODsums.values()) #sum of all the TOD sum tables 
    for x in TODsums.keys():
        TODsums[x] = TODsums[x].reset_index() #fix indices
    return TODsums


In [ ]:
#set up route to mode to mode group table
def set_up_metamode_table(b):
    '''flag each route type by metaMode'''
    routemode = pd.read_csv(b + r'Databases/Statewide_Routes_2018S.csv', usecols=["Routes_ID", "Mode"]).drop_duplicates()

    #filter if detailed
    if len(routeList) > 0:
        routemode = routemode[routemode['Routes_ID'].isin(routeList)]
    #assign metamode to routes and modes
    routemode['metaMode'] = np.where(routemode['Mode'].isin([1,2,3]), 'MBTA_Bus',
                                    np.where(routemode['Mode'].isin([5,6,7,8]), 'Heavy_Rail',
                                            np.where(routemode['Mode'].isin([4,12,13]), 'Light_Rail',
                                                    np.where(routemode['Mode'].isin([9,32,33,34,35,36,37,38,39,40]), 'Commuter_Rail',
                                                            np.where(routemode['Mode'].isin([10,11]), 'Ferry',
                                                                    np.where(routemode['Mode'].isin([14,15,16]), 'Shuttle_Express',
                                                                            np.where(routemode['Mode'].isin([41,42,43,44,17,18,19,20,21,22]), 'RTA',
                                                                                    np.where(routemode['Mode'].isin([23,24,25,26,27,28,29,30,31]), 'Private',
                                                                                            np.where(routemode['Mode'].isin([70]), 'Walk', None)))))))))
    return routemode


In [ ]:
def join_and_agg(TODSums, routemode):
    '''aggregate the on and offs by route or metaMode'''
#set the group by field depending on if standard or detailed report
    if len(routeList) > 0:
        agg = 'ROUTE'
    else: 
        agg = 'metaMode'

    for x in TODsums.keys():
        if len(routeList)> 0:
            TODsums[x] = routefile.merge(TODsums[x], how='outer', left_on='Route_ID', right_on='ROUTE')
            TODsums[x]['ROUTE'] = TODsums[x]['Route_Name'].str.replace('.', ':', regex=True).str.split(':').str[0]
        #join each table to route mode
            TODsums[x] = routemode.merge(TODsums[x], how='right', left_on='Routes_ID', right_on='Route_ID')
        else:
            TODsums[x] = routemode.merge(TODsums[x], how='right', left_on='Routes_ID', right_on='ROUTE')
        #sum all On/Off fields by metamode 
        TODsums[x] = TODsums[x].groupby([agg])[['DirectTransferOff','DirectTransferOn','DriveAccessOn','EgressOff','Off','On',
                                                'WalkAccessOn','WalkTransferOff','WalkTransferOn']].agg('sum').reset_index()
    return TODsums

In [ ]:
def plots(scen2, g):
    '''make graphs!'''
    onfdict = {}
    if len(routeList) > 0: #if detailed/standard use appropriate agg field to graph
        xVal = 'ROUTE'
    else:
        xVal = 'metaMode'
    #make faceted graph for base and comparative together    
    if 'Comparative Model' in bases.keys():
        scen2['compGraph']={}
        for tod in scen['Base Model'].keys(): #add flag field so can smush both scenario tables into one
            scen2['Base Model'][tod]['Scenario']='Base'
            scen2['Comparative Model'][tod]['Scenario']='Comparative'
            scen2['compGraph'][tod]=scen2['Base Model'][tod].append(scen2['Comparative Model'][tod]) #smoosh
            
        TODsums = scen2['compGraph']
        
        for z in TODsums.keys(): #make graphs (stacked bar)
            #set up table so can use for facets (wide to long format and flag field)
            lng = TODsums[z].drop(['DirectTransferOff','EgressOff','Off','On','WalkTransferOff'], axis = 1).melt(id_vars = [xVal, 'Scenario'], value_name = 'Count', ignore_index=False) #long to allow flag
            #lng=lng.reset_index() #Scenario will be facet field
            #make sure ids are strings for graphing purposes
            lng[xVal] = lng[xVal].astype(str)
            #make faceted stacked bar graphs (on and off dif graphs)
            on_off = px.bar(lng, x = xVal, y = 'Count', color = 'variable', facet_col = 'Scenario',title='Base and Comparative Model: '+z+' Boardings')
            #save graphs
            onfdict[z] = on_off
    else: #if only BASE
        TODsums = scen2['Base Model']
        for z in TODsums.keys(): #go through TOD
            TODsums[z][xVal] = TODsums[z][xVal].astype(str) #make safe for graphing
            onfdict[z] = px.bar(TODsums[z], x=xVal, y=['DirectTransferOn','DriveAccessOn','WalkAccessOn','WalkTransferOn'],  
                               title='Base Model '+z+' Boardings') #graph!
    return onfdict

In [ ]:
def diftab(scen):
#make difference tables
    onfdict = {}
    if len(routeList) > 0: #if detailed/standard use appropriate agg field to graph
        xVal = 'ROUTE'
    else:
        xVal = 'metaMode'
    if len(bases.keys()) ==  2:#if two scenarios
        for z in TODsums.keys(): #for each TOD
            #take the difference (and replace for TOD in the global TODsums)
            TODsums[z] = (scen['Base Model'][z].set_index(xVal).drop('Scenario', axis=1) - scen['Comparative Model'][z].set_index(xVal).drop('Scenario', axis=1)).reset_index()
            #make sure ids are strings for graphing purposes
            TODsums[z][xVal] = TODsums[z][xVal].astype(str)
            onfdict[z] = px.bar(TODsums[z], x=xVal, y=['DirectTransferOn','DriveAccessOn','WalkAccessOn','WalkTransferOn'],  
                               title='Difference in '+z+' Boardings')
    scen['Difference'] = [TODsums, onfdict] #add difference data and graphs to scen dict
    return scen

In [ ]:
#call things
#write a SUPER FUNCTION!!! (which calls all functions)
scen = {}

for g in bases.keys(): #run all these functions for each scenario
    TODsums = import_transit_assignment(bases[g]) #get the total boarding per route per TOD
    routemode = set_up_metamode_table(bases[g]) #assign modes to all routes OR restrict to just selected routes
    TODsums = join_and_agg(TODsums, routemode) #aggregate by mode or route
    
    scen[g] = TODsums
    #make graphs
scen['compGraph'] = plots(scen,g)  #package the data for showing graphs

#this is just for getting the difference to happen
if len(bases.keys())==2:
    scen = diftab(scen)


## Look at Results by TOD

In [ ]:
#Show AM Boardings
scen['compGraph']['AM'].show()
#if comparative, also show graphs of boarding differences (base - comparative) 
if len(bases.keys())==2:
    scen['Difference'][1]['AM'].show()


In [ ]:
#Show MD Boardings
scen['compGraph']['MD'].show()
#if comparative, also show graphs of boarding differences (base - comparative) 
if len(bases.keys())==2:
    scen['Difference'][1]['MD'].show()

In [ ]:
#Show PM Boardings
scen['compGraph']['PM'].show()
#if comparative, also show graphs of boarding differences (base - comparative) 
if len(bases.keys())==2:
    scen['Difference'][1]['PM'].show()

In [ ]:
#Show NT Boardings
scen['compGraph']['NT'].show()
#if comparative, also show graphs of boarding differences (base - comparative) 
if len(bases.keys())==2:
    scen['Difference'][1]['NT'].show()

In [ ]:
#Show Daily Boardings
scen['compGraph']['daily'].show()
#if comparative, also show graphs of boarding differences (base - comparative) 
if len(bases.keys())==2:
    scen['Difference'][1]['daily'].show()